# Tidal Consituent Extraction Tutorial
## Introduction

This tutorial demonstrates the use of the `hamonica` Python library to extract tidal constituent data. `harmonica` can provide the amplitude, phase, and speed of specified constituents at specified point locations. Various nodal factors of a constituent can also be obtained at a specifed time. The resolution of the datasets and the supported constituents vary depending on the tidal model used. `harmonica` currently supports the TPXO (v7-9), ADCIRC (v2015), LeProvost, and FES2014 tidal models. Resources for TPXO, ADCIRC, and LeProvost are automatically retrieved if not found (requires Internet connection). Due to licensing restrictions, the FES2014 data files must already exist locally.

## Example - Simple Points

The following example extracts tidal components of two constituents at two ocean locations. Extracting nodal factors of the constituents at a specified time is also shown.

To begin, import the necessary modules.

In [1]:
import datetime
from harmonica.tidal_constituents import Constituents

Create a list of the constituents of interest. See the classes in `resource.py` for a listing of the models' supported constituents and their expected codes. Universally compatiable constituents are used here.

In [2]:
constituents = ['M2', 'S2']

Next, construct the tidal constituent extractor object. The TPXO8 model is specified at construction here. This argument is optional and defaults to the TPXO9 model.

In [3]:
extractor = Constituents(model='tpxo8')

The next section of code demonstrates using the interface to get amplitudes, frequencies, speeds, earth tidal reduction factors, equilibrium arguments, and nodal factors for specified constituents at a specified time. The astronomical equations used to calculate these variables do not depend on the tidal model being used, so the values are always the same regardless of the extractor's current tidal model.

In [4]:
# Get astronomical nodal factor data (not dependent on the tidal model)
nodal_factors = extractor.get_nodal_factor(names=constituents, timestamp=datetime.datetime(2018, 8, 30, 15))
print(nodal_factors.to_string())

# Expected output:
#      amplitude  frequency      speed  earth_tide_reduction_factor  equilibrium_argument  nodal_factor
#  M2   0.242334   0.000141  28.984104                        0.693            345.201515      1.087974
#  S2   0.112841   0.000145  30.000000                        0.693             90.000000      1.000000

    amplitude  frequency      speed  earth_tide_reduction_factor  equilibrium_argument  nodal_factor
M2   0.242334   0.000141  28.984104                        0.693            345.201515      1.087974
S2   0.112841   0.000145  30.000000                        0.693             90.000000      1.000000


No external resources have been required up to this point. The remainder of the tutorial uses the various tidal models to extract consituent data at point locations. If the required resources do not exist locally at this time, `harmonica` will attempt to download the files from the Internet (with the exception of the FES2014 model). `__init__.py` contains a `config` dict with the default resource locations. Set the `pre_existing_data_dir` variable to change the directory `harmonica` searches for existing resources. Set the `data_dir` variable to change the directory `harmonica` saves downloaded resources. Resource files are expected to be in subfolders with the name of the model. If a required resource exists in either directory, it will not be downloaded.

Create a list of point locations where tidal harmonic components will be extracted. The first location is in the Northwest Atlantic Ocean, and the second point is in the Northeast Pacific Ocean. Locations should be specified as tuples of latitude and longitude degrees. Latitude coordinates should be in the \[-90.0, 90.0\] range. Longitude coordinates should in either the \[0.0, 360.0\] or \[-180.0, 180.0\] range.

In [5]:
# Need to be in (lat, lon), not (x, y)
locations = [
    (39.74, 285.93),   # (39.74, -74.07),
    (46.18, -124.38),  # (46.18, 235.62),
]

The next lines of code extract constituent amplitude, phase, and speed using the TPXO8 model specified at construction. A `Pandas.DataFrame` is returned for each of the requested locations. Speeds of phase change published by NOAA (https://tidesandcurrents.noaa.gov) are provided for convenience but are spatially constant. To ease comparison with other model results in this tutorial, the `positive_ph` argument has been set to `True`. This ensures all ouput phases are positive.

In [6]:
tpxo_comps = extractor.get_components(locations, constituents, positive_ph=True)
for loc, comps in zip(locations, tpxo_comps.data):
    print(loc)
    print(comps.sort_index().to_string() + '\n')

# Expected output:
# (39.74, 285.93)
#     amplitude       phase      speed
# M2   0.560701  352.265551  28.984104
# S2   0.106576   18.615143  30.000000
# 
# (46.18, -124.38)
#     amplitude       phase      speed
# M2   0.910663  234.117743  28.984104
# S2   0.253107  260.181916  30.000000

(39.74, 285.93)
    amplitude       phase      speed
M2   0.560701  352.265551  28.984104
S2   0.106576   18.615143  30.000000

(46.18, -124.38)
    amplitude       phase      speed
M2   0.910663  234.117743  28.984104
S2   0.253107  260.181916  30.000000



The tidal model can be switched with every call to `get_components` by specifying the `model` argument. Switch to the ADCIRC v2015 model and compare the results.

In [7]:
adcirc_comps = extractor.get_components(locations, constituents, model='adcirc2015')
for loc, comps in zip(locations, adcirc_comps.data):
    print(loc)
    print(comps.sort_index().to_string() + '\n')

# Expected output:
# (39.74, -74.07)
#     amplitude       phase      speed
# M2   0.554417  352.729485  28.984104
# S2   0.106317   16.530658  30.000000
# 
# (46.18, -124.38)
#     amplitude       phase      speed
# M2   0.944207  230.498719  28.984104
# S2   0.264533  256.892259  30.000000

(39.74, -74.07)
    amplitude       phase      speed
M2   0.554417  352.729485  28.984104
S2   0.106317   16.530658  30.000000

(46.18, -124.38)
    amplitude       phase      speed
M2   0.944207  230.498719  28.984104
S2   0.264533  256.892259  30.000000



Repeat the extraction using the LeProvost model.

In [8]:
leprovost_comps = extractor.get_components(locations, constituents, model='leprovost')
for loc, comps in zip(locations, leprovost_comps.data):
    print(loc)
    print(comps.sort_index().to_string() + '\n')

# Expected output:
# (39.74, -74.07)
#     amplitude       phase      speed
# M2   0.589858  353.748502  28.984104
# S2   0.083580   20.950538  30.000000
# 
# (46.18, -124.38)
#     amplitude       phase      speed
# M2   0.858488  232.029922  28.984104
# S2   0.242000  258.787649  30.000000

(39.74, -74.07)
    amplitude       phase      speed
M2   0.589858  353.748502  28.984104
S2   0.083580   20.950538  30.000000

(46.18, -124.38)
    amplitude       phase      speed
M2   0.858488  232.029922  28.984104
S2   0.242000  258.787649  30.000000



To use the FES2014 model, either change the `__init__.config` variables or copy the data files to one of the appropriate locations. Files should be in a `fes2014` subfolder of the referenced directory. See the class `FES2014Resources` in `resource.py` for a listing of expected filenames. Only the height NetCDF datasets are required.

In [9]:
fes2014_comps = extractor.get_components(locations, constituents, model='fes2014')
for loc, comps in zip(locations, fes2014_comps.data):
    print(loc)
    print(comps.sort_index().to_string() + '\n')

# Expected output:
# (39.74, 285.93)
#     amplitude       phase      speed
# M2   0.574708  354.674980  28.984104
# S2   0.111234   15.286627  30.000000
# 
# (46.18, 235.62)
#     amplitude       phase      speed
# M2   0.903397  229.333500  28.984104
# S2   0.249951  255.765785  30.000000

(39.74, 285.93)
    amplitude       phase      speed
M2   0.574708  354.674980  28.984104
S2   0.111234   15.286627  30.000000

(46.18, 235.62)
    amplitude       phase      speed
M2   0.903397  229.333500  28.984104
S2   0.249951  255.765785  30.000000

